In [4]:
import numpy as np
import pandas as pd

temp_df = pd.read_csv('IMDB Dataset.csv')
df = temp_df.iloc[:10000]

df.drop_duplicates(inplace=True)


import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text


df['review'] = df['review'].apply(remove_tags)
df['review'] = df['review'].apply(lambda x:x.lower())

C:\Users\Debdutta Chatterjee\AppData\Local\Temp\ipykernel_30820\4106985964.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)
C:\Users\Debdutta Chatterjee\AppData\Local\Temp\ipykernel_30820\4106985964.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)
C:\Users\Debdutta Chatterjee\AppData\Local\Temp\ipykernel_30820\4106985964.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See

In [7]:
!python -m nltk.downloader stopwords

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package stopwords to C:\Users\Debdutta
[nltk_data]     Chatterjee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [8]:
from nltk.corpus import stopwords
sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

C:\Users\Debdutta Chatterjee\AppData\Local\Temp\ipykernel_30820\2984342538.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [11]:
!python -m nltk.downloader punkt stopwords

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to C:\Users\Debdutta
[nltk_data]     Chatterjee\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Debdutta
[nltk_data]     Chatterjee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [16]:
!python -m nltk.downloader punkt_tab stopwords

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt_tab to C:\Users\Debdutta
[nltk_data]     Chatterjee/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to C:\Users\Debdutta
[nltk_data]     Chatterjee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
import nltk
import os
from gensim.utils import simple_preprocess

# Force download to a known location
nltk_data_dir = os.path.join(os.path.expanduser('~'), 'nltk_data')
nltk.download('punkt_tab', download_dir=nltk_data_dir, quiet=False)
nltk.download('stopwords', download_dir=nltk_data_dir, quiet=False)

# Add to path
if nltk_data_dir not in nltk.data.path:
    nltk.data.path.insert(0, nltk_data_dir)

from nltk import sent_tokenize

# Build sentence list safely
story = []
for doc in df['review'].astype(str):
    try:
        raw_sent = sent_tokenize(doc)
        for sent in raw_sent:
            story.append(simple_preprocess(sent))
    except Exception as e:
        print(f"Error processing doc: {e}")
        continue

print(f"Total sentences processed: {len(story)}")
# ...existing code...

[nltk_data] Downloading package punkt_tab to C:\Users\Debdutta
[nltk_data]     Chatterjee\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Debdutta
[nltk_data]     Chatterjee\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total sentences processed: 106789


In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [19]:
model.build_vocab(story)

In [20]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5853793, 6189340)

In [21]:
len(model.wv.index_to_key)

31807

In [22]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [23]:
document_vector(df['review'].values[0])

array([-0.16544573,  0.06248593,  0.07879287,  0.06458876,  0.2350135 ,
       -0.56824535,  0.30062285,  0.81751716, -0.21189125, -0.32854733,
       -0.0718242 , -0.44317797, -0.09458418,  0.0155193 , -0.00601026,
       -0.28963295,  0.0835732 , -0.3351056 , -0.04991894, -0.7852858 ,
        0.04017147,  0.29336888,  0.06064676, -0.19309776,  0.03956757,
       -0.04209666, -0.0933666 , -0.14241022, -0.50299096,  0.06092562,
        0.37817824, -0.20898075,  0.18556409, -0.41904026, -0.14416844,
        0.5730027 ,  0.19646838, -0.39489087, -0.1489744 , -0.62464476,
        0.0031885 , -0.21055278,  0.03750465,  0.02718301,  0.39106292,
       -0.20230947, -0.2775816 , -0.2619167 , -0.01908296,  0.21231437,
       -0.01978374, -0.36232463, -0.28523326, -0.17943223, -0.07775246,
        0.09776937,  0.17392775,  0.0123529 , -0.37073058,  0.19028771,
        0.13549542,  0.08041345,  0.27044302,  0.09804777, -0.47222704,
        0.44802266,  0.01649114,  0.13086428, -0.43127424,  0.36

In [24]:
from tqdm import tqdm
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9987/9987 [02:53<00:00, 57.67it/s]


In [26]:
X = np.array(X)

In [28]:
y = df['sentiment']

In [29]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [31]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7587587587587588